In [7]:
import xml.etree.ElementTree as ET
from shapely.geometry import Point, Polygon
import geopy.distance
import pandas as pd
import json

In [8]:
def extract_coordinates_from_kml(kml_file):
    # Parse the KML file using ElementTree
    tree = ET.parse(kml_file)
    root = tree.getroot()

    # Define the namespace used in the KML file
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Create an empty dictionary to store the coordinates
    coordinates_dict = {}

    # Loop over all the Placemark elements in the KML file
    for placemark in root.findall('.//kml:Placemark', ns):
        # Extract the name of the city from the Placemark element
        city_name = placemark.find('kml:name', ns).text

        # Extract the Polygon element from the Placemark element
        polygon = placemark.find('.//kml:Polygon', ns)

        # Extract the coordinates from the Polygon element
        coordinates_str = polygon.find('.//kml:coordinates', ns).text
        coordinates_list = coordinates_str.strip().split()

        # Split each coordinate into longitude, latitude, and altitude
        coordinates = []
        for coord_str in coordinates_list:
            coord = [float(c) for c in coord_str.split(',')][:2]
            coordinates.append(coord)

        # Add the coordinates to the dictionary, indexed by city name
        coordinates_dict[city_name] = coordinates

    return coordinates_dict


def points_in_polygon(polygon, point_coords):
    # Create a point object using the point coordinate
    point = Point(point_coords['lon'], point_coords['lat'])
    
    # Check if the point is within the polygon
    if polygon.contains(point):
        return 1
    else:
        return 0
    
def is_in_center(center, radius, point):
    return int(geopy.distance.geodesic(center, [point['lat'], point['lon']]).km < radius)

In [9]:
cities_centers_coords = extract_coordinates_from_kml('NML.kml')

with open('radii.json', 'r') as f:
    radii = json.load(f)


with open('centers.json', 'r') as f:
    centers = json.load(f)

In [10]:
cities_centers_coords

{'adelaide': [[138.577378869525, -34.90869580981371],
  [138.5790520923726, -34.91840169673218],
  [138.581435912737, -34.93081268855692],
  [138.5833590876974, -34.94161415879188],
  [138.5965549652644, -34.94110800576042],
  [138.6080949366872, -34.94050520278223],
  [138.6240222079721, -34.93967911489686],
  [138.6233483078528, -34.92820372599761],
  [138.6157404528772, -34.92292940209634],
  [138.6151782459645, -34.92149985737385],
  [138.6134881804518, -34.913759879896],
  [138.6122682564261, -34.90819330637865],
  [138.612474819054, -34.90656873241445],
  [138.6123864702961, -34.9056222228684],
  [138.6092510988896, -34.90242582917269],
  [138.5974284377484, -34.89903956453966],
  [138.5883962261448, -34.89958423254525],
  [138.577378869525, -34.90869580981371]],
 'Belfast': [[-5.921276369774107, 54.60264102034873],
  [-5.923805448316138, 54.60393220719938],
  [-5.92545044802073, 54.6058603739336],
  [-5.930798992942058, 54.60737501935248],
  [-5.934357237544362, 54.6069091113983

In [11]:
for city, center_coords in centers.items():
    city = city.lower()
    radius = radii[city]

    nodes_df = pd.read_csv(f'data/{city}/network_nodes.csv', sep=';')

    nodes_df['city_center'] = nodes_df.apply(lambda x: is_in_center(center_coords, radius, x), axis=1)
    
    print(f'City : {city}')
    print(nodes_df['city_center'].value_counts())
    print(nodes_df['city_center'].mean())
    
    print('================')

    nodes_df.to_csv(f'data/{city}/network_nodes_labeled.csv', index=False)

City : adelaide
0    5490
1    2058
Name: city_center, dtype: int64
0.2726550079491256
City : belfast
0    1565
1     352
Name: city_center, dtype: int64
0.18362023995826812
City : berlin
0    3841
1     760
Name: city_center, dtype: int64
0.16518148228645946
City : bordeaux
0    2900
1     535
Name: city_center, dtype: int64
0.15574963609898107
City : brisbane
0    7992
1    1653
Name: city_center, dtype: int64
0.1713841368584759
City : canberra
0    2307
1     457
Name: city_center, dtype: int64
0.16534008683068016
City : detroit
0    4984
1     699
Name: city_center, dtype: int64
0.12299841632940349
City : dublin
0    3994
1     577
Name: city_center, dtype: int64
0.12623058411726099
City : grenoble
0    1296
1     251
Name: city_center, dtype: int64
0.16224951519069167
City : helsinki
0    6071
1     915
Name: city_center, dtype: int64
0.13097623819066706
City : kuopio
0    470
1     79
Name: city_center, dtype: int64
0.14389799635701275
City : lisbon
0    6159
1     914
Name: city